<center>

# **Gather indicator data**

</center>

Data is downloades directly from the WHO API </br>
https://www.who.int/data/gho/info/gho-odata-api

In [45]:
import pandas as pd
from prettytable import PrettyTable
import matplotlib.pyplot as plt
import requests
import io
import os

In [46]:
#Mount your Google Drive to Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Open country codes for the Americas and the Caribbean

In [47]:
output_path = '/content/drive/MyDrive/2023_ODS_OPS/Data/'
file_name   = 'CountryCodes_Americas.xlsx'

file_path = os.path.join(output_path, file_name)


In [48]:
countries= pd.read_excel(file_path,sheet_name='Health')

#df = df.drop_duplicates(subset='Country Name')
countries.head()
#len(countries["Country Code"].unique())

,Country Name,Code
0,Antigua and Barbuda,ATG
1,Argentina,ARG
2,"Bahamas, The",BHS
3,Barbados,BRB
4,Belize,BLZ


In [11]:
countries_code = countries['Code'].tolist()

## **Retrieving the indicators list**

This code give us the list of indicators directly from  WHO (API)

In [49]:
url = "https://ghoapi.azureedge.net/api/Indicator"

try:
    response = requests.get(url)
    response.raise_for_status()

    indicator_data = response.json()["value"]

    indicators = []
    for indicator in indicator_data:
        indicators.append({
            "Indicator Name": indicator["IndicatorName"],
            "Indicator Code": indicator["IndicatorCode"]
        })

    df = pd.DataFrame(indicators)
    print(df)
except requests.exceptions.RequestException as e:
    print("Error occurred:", e)


                                         Indicator Name  \
0     Ambient air pollution  attributable DALYs per ...   
1           Household air pollution attributable deaths   
2     Household air pollution attributable deaths in...   
3            Household air pollution attributable DALYs   
4     Household air pollution attributable DALYs in ...   
...                                                 ...   
2340  Total expenditure on dental healthcare in mill...   
2341             Number of isolates tested for cefixime   
2342        Total population in thousands (oral health)   
2343  Best-practice policy implemented for industria...   
2344  General availability of urine strips for gluco...   

                             Indicator Code  
0                                    AIR_10  
1                                    AIR_11  
2                                    AIR_12  
3                                    AIR_15  
4                                    AIR_16  
...            

In [ ]:
output_path = '/content/drive/MyDrive/2023_ODS_OPS/Data/'
file_name   = 'WHO_List_of_indicators_and_codes.xlsx'

output_file = os.path.join(output_path, file_name)

df.to_excel(output_file, index=False)

## **Retrieving indicator data**

Download an indicator by specifying the indicator code. This will return all associated data for that specific indicator.

Available dimenstions http://ghoapi.azureedge.net/api/WHOSIS_000001

In [50]:
def get_indicator_data(indicator_code):
    url = f"https://ghoapi.azureedge.net/api/{indicator_code}"
    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()["value"]

        # Create empty lists to store the data
        IDs = []
        IndicatorCodes = []
        Countries = []
        years  = []
        values = []
        sex    = []
        Date1  = []
        Date2  = []


        # Extract the data and populate the lists
        for item in data:
            Id = item["Id"]
            IndicatorCode = item["IndicatorCode"]
            year          = item["TimeDimensionValue"]
            value         = item["NumericValue"]
            Country       = item["SpatialDim"]
            Sexo          = item["Dim1"]
            Date11        = item["TimeDimensionBegin"]
            Date22        = item["TimeDimensionEnd"]

            IDs.append(Id)
            IndicatorCodes.append(IndicatorCode)
            Countries.append(Country)
            years.append(year)
            values.append(value)
            sex.append(Sexo)
            Date1.append(Date11)
            Date2.append(Date22)

        # Create the dataframe
        df = pd.DataFrame({"Id": IDs,"SEX": sex, "IndicatorCode": IndicatorCodes, "Year": years, "Date1": Date1,"Date2":Date2,"Value": values, "Country": Countries})

        return df

    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)



# **UHC Service Coverage Index**

defined as the average coverage of essential services based on tracer interventions that include reproductive, maternal, newborn and child health, infectious diseases, non-communicable diseases and service capacity and access, among the general and the most disadvantaged population.

The indicator is an index reported on a unitless scale of 0 to 100, which is computed as the geometric mean of 14 tracer indicators of health service coverage. The tracer indicators are as follows, organized by four components of service coverage: 1. Reproductive, maternal, newborn and child health 2. Infectious diseases 3. Noncommunicable diseases 4. Service capacity and access

In [14]:
#https://www.who.int/data/gho/data/themes/topics/service-coverage

#Primary data availability for UHC Service Coverage Index (SDG 3.8.1) (%)
Healch_coverage_index = "UHC_AVAILABILITY_SCORE"

In [15]:
# Call the function with the URL
df = get_indicator_data(Healch_coverage_index)

## Filter countries in the Americas and the Caribbean
Americas_info = df[df['Country'].isin(countries_code)]
Americas_info.head(5)

Americas_info.to_excel('/content/drive/MyDrive/2023_ODS_OPS/HealthCoverage.xlsx')


## **Selected Indiators**

In [55]:
#List of selected indicators
path_selected_indicators ='/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/1.List_of_Selected_Health_Coverage_Indicators.xlsx'
SheetName = '2_Selected_Health_Coverage'

selected_indicators = pd.read_excel(path_selected_indicators,sheet_name=SheetName )
selected_index_code = selected_indicators['IndicatorCode']

#Revisar la lista de excel para ver el numero hasta el que se deben considerar
selected_index_code = [selected_index_code[i] for i in range(0,12)]

['UHC_INDEX_REPORTED',
 'GHED_CHEGDP_SHA2011',
 'GHED_GGHE-DCHE_SHA2011',
 'GHED_GGHE-DGGE_SHA2011',
 'GHED_GGHE-D_pc_US_SHA2011',
 'GHED_CHE_pc_US_SHA2011',
 'GHED_PVT-D_pc_US_SHA2011',
 'GHED_GGHE-DGDP_SHA2011',
 'GHED_PVT-DCHE_SHA2011',
 'GHED_EXTCHE_SHA2011',
 'GHED_OOPSCHE_SHA2011',
 'UHC_SCI_CAPACITY']

In [56]:
index_code = selected_index_code[1]
df         = get_indicator_data(index_code)

## Filter countries in the Americas and the Caribbean
Americas_info = df[df['Country'].isin(countries_code)]
Americas_info.head(5)

df['Country'].unique()

array(['AFG', 'ALB', 'DZA', 'AND', 'AGO', 'ATG', 'ARG', 'ARM', 'AUS',
       'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ',
       'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR', 'BFA',
       'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CAF', 'TCD', 'CHL', 'CHN',
       'COL', 'COM', 'COG', 'COK', 'CRI', 'HRV', 'CUB', 'CYP', 'CZE',
       'CIV', 'COD', 'DNK', 'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'SLV',
       'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'FJI', 'FIN', 'FRA', 'GAB',
       'GMB', 'GEO', 'DEU', 'GHA', 'GRC', 'GRD', 'GTM', 'GIN', 'GNB',
       'GUY', 'HTI', 'HND', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ',
       'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR',
       'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LTU',
       'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MHL', 'MRT',
       'MUS', 'MEX', 'FSM', 'MCO', 'MNG', 'MNE', 'MAR', 'MOZ', 'MMR',
       'NAM', 'NRU', 'NPL', 'NLD', 'NZL', 'NIC', 'NER', 'NGA', 'NIU',
       'NOR', 'OMN',

In [59]:
##Filtrar solo informacion para ambos sexos
Americas_info = Americas_info[Americas_info['SEX']=='BTSX']

In [64]:
Americas_info['Year'].unique()

array(['2013', '2010'], dtype=object)

## Identify the missing countries

In [38]:
def find_unique_values(list1, list2):
  set1 = set(list1)
  set2 = set(list2)
  unique_values = list(set1.symmetric_difference(set2))
  return unique_values

unique_countries = Americas_info['Country'].unique()

result = find_unique_values(unique_countries, countries_code)
print('Total countries with data',len(unique_countries))

countries_name = countries[countries['Code'].isin(result)]
print(countries_name['Country Name'].tolist())


Total countries with data 27


KeyError: ignored

# Join all indicators

In [57]:
# then concatenate into a single DataFrame
americas_dfs = []

for index_code in selected_index_code:
    df = get_indicator_data(index_code)

    Americas_info = df[df['Country'].isin(countries_code)]
    americas_dfs.append(Americas_info)

# Concatenate the DataFrames into a single DataFrame
combined_df = pd.concat(americas_dfs, ignore_index=True)

# Filter data between 2000 and 2021 using the "Year" column
combined_df['Year'] = pd.to_numeric(combined_df['Year'])  # Convert the 'Year' column to numeric (in case it's not already)

# Filtering the data
filtered_df = combined_df[(combined_df['Year'] >= 2000) & (combined_df['Year'] <= 2021)]


In [42]:
# Replace both 'nan' and empty string '' with None (missing value) in the 'SEX' column
filtered_df['SEX'] = filtered_df['SEX'].replace(['nan', ''], None)

filtered_df['SEX'].unique()

<ipython-input-42-afc24dac4a24>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['SEX'] = filtered_df['SEX'].replace(['nan', ''], None)


array([None, 'RUR', 'TOTL', 'URB', 'FMLE', 'MLE'], dtype=object)

In [58]:
filtered_df.to_excel('/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/2.Values_Health_Coverage_Indicators_WHO.xlsx')